In [1]:
from random import randint
from numpy import array
import numpy as np
from numpy import argmax
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
import matplotlib.pyplot as plt
from keras.optimizers import Adam

%matplotlib inline

Using TensorFlow backend.


# Data

In [2]:
data = []
with open('D:\IITM\cdn1_500m_sigmetrics18.tr' , errors='ignore' ) as fin:
    fin.seek(0)
    data = fin.read(400000000) 
len(data)

400000000

In [4]:
data[399999985:400000000]

'2198\n16561205 9'

In [5]:
# Our dataset is large sequence in form of the string and each request segment is separated by \n.
# Each segment contains request ID and size of the requested file
# Here we will separate each string by ' ' and '\n'
r = []
i = j = 0
while j < 399999990:
    x = []
    while data[i] != ' ' and data[i] != '\n':
        if i%100000000 == 0:
            print(i)
        x.append(data[i]) 
        i += 1
    r.append(x)
    j = i + 1
    i = j

0
100000000
200000000
300000000


In [6]:
len(r)

49683612

In [7]:
# r contains sequence od number of request, ID and size. so we will make an array to arrange ID and size.
i = 0
j = 1
re = []

while i < len(r):
    temp = []
    while(j%3 != 0) and j < len(r):
        temp.append(r[j])
        j += 1
    re.append(temp)
    j = j+1
    i = j
r = []       

In [8]:
re = np.array(re)
re.shape

(16561204, 2)

In [9]:
# to convert string into real numbers 
data = []
for i in range(len(re)):
    temp = []
    for j in range(2):
        sum = 0
        for k in range(len(re[i][j])):
            sum = sum + int(re[i][j][k]) * pow(10,len(re[i][j])-k-1)
        temp.append(sum)
    data.append(temp) 
re = []

In [67]:
data = np.array(data)
data.shape

(16561204, 2)

In [68]:
req = data[:,0]
min = np.min(req)
max = np.max(req)
min,max

(454, 4294966645)

In [69]:
id = np.unique(req)
len(id)

3069207